In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import random
import numpy as np
import optuna
from optuna.pruners import MedianPruner
from scipy.signal import butter, sosfiltfilt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.feature_selection import mutual_info_classif
from sklearn.base import BaseEstimator, ClassifierMixin
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from modules.competition_dataset import EEGDataset
from pyriemann.utils.mean import mean_riemann
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, classification_report
from scipy.linalg import expm, logm, inv
from scipy.optimize import minimize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data_path = './data/mtcaic3'
lda_model_path = './checkpoints/mi/models/lda_mi.pkl'

# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_random_seeds(42)

In [ ]:
# -----------------------------------------------------------------------------
# 1) RSFTransformer that learns W_opt using BFGS
# -----------------------------------------------------------------------------
class RSFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, d=8, eps=1e-6, maxiter=500):
        self.d = d
        self.eps = eps
        self.maxiter = maxiter

    def fit(self, X, y):
        covs = Covariances(estimator="lwf").transform(X)
        C1 = mean_riemann(covs[y == 0])
        C2 = mean_riemann(covs[y == 1])
        Nc = C1.shape[0]
        W0 = np.random.randn(Nc, self.d)

        def J(w_flat):
            W = w_flat.reshape(Nc, self.d)
            S1 = W.T @ C1 @ W + np.eye(self.d) * self.eps
            S2 = W.T @ C2 @ W + np.eye(self.d) * self.eps
            return np.linalg.norm(logm(inv(S1) @ S2), "fro") ** 2

        def grad(w_flat):
            W = w_flat.reshape(Nc, self.d)
            S1 = W.T @ C1 @ W + np.eye(self.d) * self.eps
            S2 = W.T @ C2 @ W + np.eye(self.d) * self.eps
            A = inv(S1) @ S2
            L = logm(A)
            t1 = 2 * C1 @ W @ inv(S1) @ L @ inv(S1)
            t2 = 2 * C2 @ W @ inv(S2) @ L @ inv(S2)
            return (t1 - t2).ravel()

        res = minimize(lambda w: -J(w), W0.ravel(), jac=lambda w: -grad(w), method="BFGS", options={"maxiter": self.maxiter})
        self.W_opt = res.x.reshape(Nc, self.d)
        return self

    def transform(self, X):
        return np.einsum("cd,ect->edt", self.W_opt, X)

class FilterBankRTSClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, bands, fs, order, C, kernel, gamma, degree, coef0, probability):
        self.bands = bands if bands else [(8, 12), (12, 16), (16, 20), (20, 24), (24, 30)]
        self.fs = fs
        self.order = order
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.probability = probability

    def compute_fb_covs(self, X):
        """X: (n_trials, C, T) → fb_covs: (n_trials, B, C, C)"""
        # Pre-compute SOS filters if not done
        if not hasattr(self, "sos_bands"):
            self.sos_bands = [butter(self.order, (l / (self.fs / 2), h / (self.fs / 2)), btype="bandpass", output="sos") for l, h in self.bands]

        n, C, _ = X.shape
        B = len(self.sos_bands)
        fb_covs = np.zeros((n, B, C, C))
        for i, sos in enumerate(self.sos_bands):
            Xf = sosfiltfilt(sos, X, axis=2)
            fb_covs[:, i] = Covariances(estimator="lwf").transform(Xf)
        return fb_covs

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        # Flatten for tangent space
        covs_flat = fb_covs.reshape(n * B, C, C)
        labels_rep = np.repeat(y, B)

        # Fit tangent space
        self.ts = TangentSpace(metric="riemann").fit(covs_flat, labels_rep)
        Z = self.ts.transform(covs_flat)
        Z = Z.reshape(n, B, -1)

        # Compute mutual information weights
        self.w = mutual_info_classif(Z.reshape(n, -1), y, discrete_features=False).reshape(B, -1).mean(axis=1)
        self.w = self.w / self.w.sum()

        # Weight features
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        # Train classifier
        self.clf = make_pipeline(
            StandardScaler(),
            SVC(
                C=self.C,
                kernel=self.kernel,
                gamma=self.gamma,
                degree=self.degree,
                coef0=self.coef0,
                class_weight='balanced',
                probability=self.probability,
                random_state=42,
            ),
        )
        self.clf.fit(Z_weighted, y)
        return self

    def predict(self, X):
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        covs_flat = fb_covs.reshape(n * B, C, C)
        Z = self.ts.transform(covs_flat).reshape(n, B, -1)
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        return self.clf.predict(Z_weighted)

    def predict_proba(self, X):
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        covs_flat = fb_covs.reshape(n * B, C, C)
        Z = self.ts.transform(covs_flat).reshape(n, B, -1)
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        return self.clf.predict_proba(Z_weighted)


def load_eeg_data(data_path, window_length, stride, tmin, eeg_channels):
    ds = EEGDataset(
        data_path,
        window_length=window_length,
        stride=stride,
        task="mi",
        split="train",
        data_fraction=0.4,
        tmin=tmin,
        eeg_channels=eeg_channels,
    )
    X = np.stack([x.numpy() for x, _ in ds])
    y = np.array([label[0] for _, label in ds])
    return X, y


# Optuna optimization
data_path = "./data/mtcaic3"
cv_folds = 3

window_lengths = [250, 500, 1000, 1500]
strides = [250, 500, 750]
tmins = [0, 250, 500]


def objective(trial):
    # --- Data and channel selection parameters (same as before) ---
    wl = trial.suggest_categorical("window_length", window_lengths)
    st = trial.suggest_categorical("stride", strides)
    t0 = trial.suggest_categorical("tmin", tmins)
    all_channels = ["FZ", "C3", "CZ", "C4", "PZ", "PO7", "OZ", "PO8"]
    active_mask = [trial.suggest_int(f"ch_{ch}", 0, 1) for ch in all_channels]
    chs = [ch for ch, enabled in zip(all_channels, active_mask) if enabled]
    if len(chs) < 2:
        return 0.0

    # --- NEW: Hyperparameter for the RSFTransformer ---
    # The dimension 'd' must be between 2 and the number of selected channels.
    d_rsf = trial.suggest_int("rsf_d", 2, len(chs))

    # --- Filter bank parameters (same as before) ---
    n_bands = trial.suggest_int("n_bands", 3, 8)
    min_freq = trial.suggest_int("min_freq", 4, 12)
    max_freq = trial.suggest_int("max_freq", 25, 40)
    freq_step = (max_freq - min_freq) / n_bands
    bands = [(min_freq + i * freq_step, min_freq + (i + 1) * freq_step) for i in range(n_bands)]
    filter_order = trial.suggest_int("filter_order", 3, 6)
    fs = trial.suggest_categorical("fs", [125, 250, 500])

    # --- SVM parameters ---
    C = trial.suggest_float("C", 1e-3, 1e3, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"])
    gamma = trial.suggest_categorical("gamma", ["scale", "auto"])
    degree = trial.suggest_int("degree", 2, 5)  # only for 'poly'
    coef0 = trial.suggest_uniform("coef0", 0.0, 1.0)

    print(f"\n→ Trying: wl={wl}, st={st}, tmin={t0}, chans={chs}")

    try:
        X, y = load_eeg_data(data_path, wl, st, t0, chs)
    except Exception as e:
        print("Data loading failed:", e)
        return 0.0

    # --- CORE CHANGE: Build the pipeline ---
    pipeline = make_pipeline(
            RSFTransformer(d=d_rsf),
            FilterBankRTSClassifier(
                bands=bands,
                fs=fs,
                order=filter_order,
                C=C,
                kernel=kernel,
                gamma=gamma,
                degree=degree,
                coef0=coef0,
                probability=True,
            ),
        )

    cv = StratifiedKFold(cv_folds, shuffle=True, random_state=42)
    
    # Run cross-validation on the entire pipeline
    scores = cross_validate(pipeline, X, y, cv=cv, scoring="accuracy", return_train_score=True)

    train_acc = scores["train_score"].mean()
    val_acc = scores["test_score"].mean()

    print(f"   → Train acc: {train_acc:.3f} | Val acc: {val_acc:.3f}")
    trial.set_user_attr("train_acc", train_acc)

    return val_acc
    
    
study = optuna.create_study(direction="maximize", pruner=MedianPruner())
study.optimize(objective, n_trials=100, timeout=7200)

print("\n=== Best trial ===")
best = study.best_trial
print("Val Acc:", best.value)
print("Train Acc:", best.user_attrs["train_acc"])
print("Params:")
for k, v in best.params.items():
    print(f"  {k}: {v}")

[I 2025-06-28 17:28:58,187] A new study created in memory with name: no-name-e27acc82-92fa-4b93-9258-090bf05348df
/tmp/ipykernel_8343/2636314267.py:179: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  coef0 = trial.suggest_uniform("coef0", 0.0, 1.0)



→ Trying: wl=1500, st=250, tmin=500, chans=['FZ', 'C3', 'PZ', 'PO7', 'PO8']
task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples


[I 2025-06-28 17:29:04,979] Trial 0 finished with value: 0.0 and parameters: {'window_length': 1500, 'stride': 250, 'tmin': 500, 'ch_FZ': 1, 'ch_C3': 1, 'ch_CZ': 0, 'ch_C4': 0, 'ch_PZ': 1, 'ch_PO7': 1, 'ch_OZ': 0, 'ch_PO8': 1, 'rsf_d': 2, 'n_bands': 8, 'min_freq': 5, 'max_freq': 32, 'filter_order': 5, 'fs': 500, 'C': 9.700229111364788, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2, 'coef0': 0.3595978872096106}. Best is trial 0 with value: 0.0.


skipped: 960/960
Data loading failed: need at least one array to concatenate

→ Trying: wl=250, st=250, tmin=500, chans=['CZ', 'PO7', 'PO8']
task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples


/tmp/ipykernel_8343/2636314267.py:179: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  coef0 = trial.suggest_uniform("coef0", 0.0, 1.0)


...

skipped: 3/960


[I 2025-06-28 17:37:30,576] Trial 8 finished with value: 0.5071056823863797 and parameters: {'window_length': 500, 'stride': 750, 'tmin': 0, 'ch_FZ': 1, 'ch_C3': 0, 'ch_CZ': 1, 'ch_C4': 1, 'ch_PZ': 0, 'ch_PO7': 1, 'ch_OZ': 1, 'ch_PO8': 0, 'rsf_d': 4, 'n_bands': 7, 'min_freq': 11, 'max_freq': 31, 'filter_order': 6, 'fs': 500, 'C': 0.37469840353692024, 'kernel': 'poly', 'gamma': 'auto', 'degree': 5, 'coef0': 0.8200651483831887}. Best is trial 1 with value: 0.5314573409847582.


   → Train acc: 0.926 | Val acc: 0.507

→ Trying: wl=1000, st=250, tmin=0, chans=['FZ', 'C3', 'PZ', 'PO7']
task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples


/tmp/ipykernel_8343/2636314267.py:179: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  coef0 = trial.suggest_uniform("coef0", 0.0, 1.0)


skipped: 11/960


In [ ]:
# Trial 7 finished with value: 0.5871394563472291 and parameters: {'window_length': 250, 'stride': 500, 'tmin': 500, 'ch_FZ': 0, 'ch_C3': 0, 'ch_CZ': 1, 'ch_C4': 1, 'ch_PZ': 0, 'ch_PO7': 1, 'ch_OZ': 1, 'ch_PO8': 1, 'rsf_d': 5, 'n_bands': 3, 'min_freq': 4, 'max_freq': 39, 'filter_order': 3, 'fs': 125, 'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 7 with value: 0.5871394563472291.

class FilterBankRTSClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, bands=None, fs=250, order=4, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features="sqrt", class_weight="balanced", n_jobs=-1):
        self.bands = bands if bands else [(8, 12), (12, 16), (16, 20), (20, 24), (24, 30)]
        self.fs = fs
        self.order = order
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.class_weight = class_weight
        self.n_jobs = n_jobs

    def compute_fb_covs(self, X):
        """X: (n_trials, C, T) → fb_covs: (n_trials, B, C, C)"""
        # Pre-compute SOS filters if not done
        if not hasattr(self, "sos_bands"):
            self.sos_bands = [butter(self.order, (l / (self.fs / 2), h / (self.fs / 2)), btype="bandpass", output="sos") for l, h in self.bands]

        n, C, _ = X.shape
        B = len(self.sos_bands)
        fb_covs = np.zeros((n, B, C, C))
        for i, sos in enumerate(self.sos_bands):
            Xf = sosfiltfilt(sos, X, axis=2)
            fb_covs[:, i] = Covariances(estimator="lwf").transform(Xf)
        return fb_covs

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        # Flatten for tangent space
        covs_flat = fb_covs.reshape(n * B, C, C)
        labels_rep = np.repeat(y, B)

        # Fit tangent space
        self.ts = TangentSpace(metric="riemann").fit(covs_flat, labels_rep)
        Z = self.ts.transform(covs_flat)
        Z = Z.reshape(n, B, -1)

        # Compute mutual information weights
        self.w = mutual_info_classif(Z.reshape(n, -1), y, discrete_features=False).reshape(B, -1).mean(axis=1)
        self.w = self.w / self.w.sum()

        # Weight features
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        # Train classifier
        self.clf = make_pipeline(
            StandardScaler(),
            RandomForestClassifier(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
                class_weight=self.class_weight,
                n_jobs=self.n_jobs,
                random_state=42,
            ),
        )
        self.clf.fit(Z_weighted, y)
        return self

    def predict(self, X):
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        covs_flat = fb_covs.reshape(n * B, C, C)
        Z = self.ts.transform(covs_flat).reshape(n, B, -1)
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        return self.clf.predict(Z_weighted)

    def predict_proba(self, X):
        fb_covs = self.compute_fb_covs(X)
        n, B, C, _ = fb_covs.shape

        covs_flat = fb_covs.reshape(n * B, C, C)
        Z = self.ts.transform(covs_flat).reshape(n, B, -1)
        Z_weighted = np.concatenate([np.sqrt(self.w[i]) * Z[:, i, :] for i in range(B)], axis=1)

        return self.clf.predict_proba(Z_weighted)

class RSFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, d=8, eps=1e-6, maxiter=500):
        self.d = d
        self.eps = eps
        self.maxiter = maxiter

    def fit(self, X, y):
        covs = Covariances(estimator="lwf").transform(X)
        C1 = mean_riemann(covs[y == 0])
        C2 = mean_riemann(covs[y == 1])
        Nc = C1.shape[0]
        W0 = np.random.randn(Nc, self.d)

        def J(w_flat):
            W = w_flat.reshape(Nc, self.d)
            S1 = W.T @ C1 @ W + np.eye(self.d) * self.eps
            S2 = W.T @ C2 @ W + np.eye(self.d) * self.eps
            return np.linalg.norm(logm(inv(S1) @ S2), "fro") ** 2

        def grad(w_flat):
            W = w_flat.reshape(Nc, self.d)
            S1 = W.T @ C1 @ W + np.eye(self.d) * self.eps
            S2 = W.T @ C2 @ W + np.eye(self.d) * self.eps
            A = inv(S1) @ S2
            L = logm(A)
            t1 = 2 * C1 @ W @ inv(S1) @ L @ inv(S1)
            t2 = 2 * C2 @ W @ inv(S2) @ L @ inv(S2)
            return (t1 - t2).ravel()

        res = minimize(lambda w: -J(w), W0.ravel(), jac=lambda w: -grad(w), method="BFGS", options={"maxiter": self.maxiter})
        self.W_opt = res.x.reshape(Nc, self.d)
        return self

    def transform(self, X):
        return np.einsum("cd,ect->edt", self.W_opt, X)


window_length = 250
stride = 500
tmin = 500
eeg_channels = ['CZ', 'C4', 'PO7', 'OZ', 'PO8']
d_rsf = 5
n_bands = 3
min_freq = 4
max_freq = 39
filter_order = 3
fs = 125
n_estimators = 150
max_depth = 10
min_samples_split = 5
min_samples_leaf = 1
max_features = 'sqrt'
data_path = "./data/mtcaic3"


# Load data with besjt parameters
ds_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    task="mi",
    split="train",
    data_fraction=1,
    tmin=tmin,
    eeg_channels=eeg_channels,
)
X_train = np.stack([x.numpy() for x, _ in ds_train])
y_train = np.array([label[0] for _, label in ds_train])

# Load data with besjt parameters
freq_step = (max_freq - min_freq) / n_bands
bands = [(min_freq + i * freq_step, min_freq + (i + 1) * freq_step) for i in range(n_bands)]
ds_val = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    task="mi",
    split="train",
    data_fraction=1,
    tmin=tmin,
    eeg_channels=eeg_channels,
)
X_val = np.stack([x.numpy() for x, _ in ds_val])
y_val = np.array([label[0] for _, label in ds_val])

freq_step = (max_freq - min_freq) / n_bands
bands = [(min_freq + i * freq_step, min_freq + (i + 1) * freq_step) for i in range(n_bands)]

# Create frequency bands

# Create FilterBank classifier with best parameters
clf = make_pipeline(
    # This single object now contains your entire workflow.
        RSFTransformer(d=d_rsf),
        FilterBankRTSClassifier(
            bands=bands,
            fs=fs,
            order=filter_order,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            class_weight="balanced",
            n_jobs=-1
        )
    )

# Fit on training data
clf.fit(X_train, y_train)

# Calculate accuracy
y_pred = clf.predict(X_val)
val_acc = accuracy_score(y_val, y_pred)

print(f"Validation accuracy: {val_acc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

task: mi, split: train, domain: time, data_fraction: 1
skipped: 21/2400
task: mi, split: train, domain: time, data_fraction: 1
skipped: 21/2400
Validation accuracy: 0.5602

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.61      0.58      3317
           1       0.57      0.51      0.54      3374

    accuracy                           0.56      6691
   macro avg       0.56      0.56      0.56      6691
weighted avg       0.56      0.56      0.56      6691

